In [2]:
from typing import List, Any
import pandas as pd
import os

In [3]:
TEMPLATE = "INSERT INTO TABLE_NAME (COLUMNS) VALUES\nDATA;"

In [264]:
def transform_item(i: Any):
    if type(i) == str:
        return f"'{i}'"
    elif i == None:
        return 'NULL'
    return str(i)

def transform_list(l: List):
    return list(map(transform_item, l))

def transform_list_to_str(l: List):
    tmp = map(lambda l_: ', '.join(l_), l)
    tmp = map(lambda x: f'({x})', tmp)
    return ',\n'.join(tmp)

# print(transform_list_to_str([["'string'", '2', '3.4'], ["'string2'", '3', '4.2']]))

In [6]:
def generate_insert_tuples(table_name: str, columns: List, values: List[List[Any]]):
    insert = TEMPLATE.replace('TABLE_NAME', table_name)
    
    if len(columns) <= 0:
        insert = insert.replace('(COLUMNS) ', '')
    else:
        insert = insert.replace('COLUMNS', ', '.join(columns))
        
    values_ = map(transform_list, values)
    values_ = transform_list_to_str(values_)
    
    return insert.replace('DATA', values_)

print(generate_insert_tuples('ANIME', ['teste', 'a'], [[1, 'b'], [2, 'c']]))

INSERT INTO ANIME (teste, a) VALUES
(1, 'b'),
(2, 'c');


In [7]:
def get_unique_values(df: 'pd.DataFrame', column):
    values = set()
    for i in df[column]:
        for name in i.split(', '):
            values.add(name)
    return [[i] for i in list(values)]

In [8]:
def id_gen():
    i = 0
    while True:
        i += 1
        yield i

def get_ids(data: List[List[Any]]):
    id_ = id_gen()
    return [[i[0], next(id_)] for i in data]

In [9]:
def create_pairs(df, data, column):
    data_dict = dict(data)
    list_ = list()
    pair = zip(df['MAL_ID'], df[column])
    for anime_id, d in pair:
        dl = d.split(', ')
        for i in dl:
            list_.append([anime_id, data_dict[i]])
    return list_

In [161]:
TRANSLATE = {
    'Spring': 'primavera',
    'Winter': 'inverno',
    'Summer': 'verao',
    'Fall': 'outono'
}
def season_split(x):
    s = x.split()
    if len(s) < 2 or s[0] not in ('Spring', 'Summer', 'Winter', 'Fall'):
        return None
    else:
        return TRANSLATE[s[0]]
    
def year_split(x):
    y = x.split()
    if len(y) < 2:
        return None
    else:
        return y[1]

In [10]:
os.listdir('anime')

['html folder',
 'anime.csv',
 'rating_complete.csv',
 'anime_with_synopsis.csv',
 'watching_status.csv',
 'animelist.csv']

In [292]:
df = pd.read_csv('anime/anime.csv')

In [293]:
df['Producers'] = df['Producers'].apply(lambda s: s.replace("'", ' '))
df['Licensors'] = df['Licensors'].apply(lambda s: s.replace("'", ' '))
df['Name'] = df['Name'].apply(lambda n: n.replace("'", " "))
df['Studios'] = df['Studios'].apply(lambda s: s.replace("'", ' '))

In [294]:
genres = get_ids(get_unique_values(df, 'Genres'))
producers = get_ids(get_unique_values(df, 'Producers'))
licensors = get_ids(get_unique_values(df, 'Licensors'))
studios = get_ids(get_unique_values(df, 'Studios'))
anime_genres = create_pairs(df, genres, 'Genres')
anime_producers = create_pairs(df, producers, 'Producers')
anime_licensors = create_pairs(df, licensors, 'Licensors')
anime_studios = create_pairs(df, studios, 'Studios')

data_ = [
    ('GENERO', ['nome', 'id'], genres), 
    ('PRODUTOR', ['nome', 'id'], producers),
    ('LICENCIADOR', ['nome', 'id'], licensors),
    ('ESTUDIO', ['nome', 'id'], studios),
    ('participa', ['anime', 'genero'], anime_genres),
    ('produz', ['anime', 'produtor'], anime_producers), 
    ('licencia', ['anime', 'licenciador'], anime_licensors), 
    ('anima', ['anime', 'estudio'], anime_studios)]

In [295]:
dfa = df
dfa.drop(['Genres', 'Producers', 'Licensors', 'Studios'], axis=1, inplace=True)
dfa.drop(['English name', 'Japanese name'], axis=1, inplace=True)
dfa.drop(['Aired'], axis=1, inplace=True)

In [296]:
dfa['year'] = dfa['Premiered'].apply(year_split)
dfa['season'] = dfa['Premiered'].apply(season_split)
dfa.drop(['Premiered'], axis=1, inplace=True)

In [297]:
dfa['season'].unique()

array(['primavera', None, 'verao', 'outono', 'inverno'], dtype=object)

In [298]:
dfa['Score'] = dfa['Score'].replace(['Unknown'], 0)
dfa['Type'] = dfa['Type'].replace(['Unknown'], None)
dfa['Episodes'] = dfa['Episodes'].replace(['Unknown'], 0)
dfa['Source'] = dfa['Source'].replace(['Unknown'], 'Other')
dfa['Ranked'] = dfa['Ranked'].replace(['Unknown'], 0)

In [299]:
dfa.columns

Index(['MAL_ID', 'Name', 'Score', 'Type', 'Episodes', 'Source', 'Duration',
       'Rating', 'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching',
       'Completed', 'On-Hold', 'Dropped', 'Plan to Watch', 'Score-10',
       'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4',
       'Score-3', 'Score-2', 'Score-1', 'year', 'season'],
      dtype='object')

In [300]:
for i in range(1, 11):
    name = f'Score-{i}'
    dfa[name] = dfa[name].replace(['Unknown'], 0)

In [301]:
dfa.drop(['Duration'], axis=1, inplace=True)
dfa.drop(['Rating'], axis=1, inplace=True)
dfa.drop(['Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch'], axis=1, inplace=True)

In [302]:
table_columns = ['id', 'nome', 'nota', 'tipo', 'num_episodios', 'material_original',
                '`rank`', 'popularidade', 'num_membros', 'num_favoritos', 'num_nota10',
                'num_nota9', 'num_nota8', 'num_nota7', 'num_nota6', 'num_nota5',
                'num_nota4', 'num_nota3', 'num_nota2', 'num_nota1', 'ano', 'estacao']

In [303]:
columns = list(dfa.columns)
animes_ = zip(*[dfa[i] for i in columns])

In [304]:
data_ = [('ANIME', table_columns, animes_), *data_]

In [305]:
ws = pd.read_csv('anime/watching_status.csv')
l = zip(ws['status'], ws[' description'])
# acompanhamento
watching_status = [[i, j] for i, j in l]

data_ = [('ACOMPANHAMENTO', ['id', 'description'], watching_status), *data_]

In [279]:
# dfw = pd.read_csv('anime/animelist.csv')
# users = [[i] for i in dfw['user_id'].unique()]

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [259]:
# dfw.columns

Index(['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes'], dtype='object')

In [306]:
users = set()
    
with open('anime/animelist.csv', 'r') as fd:
    fd.readline()
    for i in fd:
        users.add(i.split(',')[0])

In [307]:
users_ = [[i] for i in users]

In [308]:
data_ = [('USUARIO', ['id'], users_), *data_]

In [309]:
with open('popula.sql', 'w+') as fd:
    for i in data_:
        fd.write(generate_insert_tuples(*i))
        fd.write('\n\n')

In [291]:
# out = open('popula.sql', 'w+')
# out.write('INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES\n')
# with open('anime/animelist.csv', 'r') as fd:
#     fd.readline()
#     last = f'({fd.readline().strip()})'
#     for i in fd:
#         out.write(f'INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES {last};\n')
#         last = f'({i.strip()})'
#     out.write(f'INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES {last};\n')
# out.close()

In [311]:
out = open('popula.sql', 'w+')
out.write('INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES\n')
limit = 0
with open('anime/animelist.csv', 'r') as fd:
    fd.readline()
    last = f'({fd.readline().strip()})'
    for i in fd:
        if limit > 500000:
            break
        out.write(f'{last},\n')
        last = f'({i.strip()})'
        limit += 1
    out.write(f'{last};\n\n')
out.close()

In [185]:
# x = zip(dfw['user_id'], dfw['anime_id'], dfw['rating'], dfw['watching_status'], dfw['watched_episodes'])
# anime_watch_data = [[*i] for i in x]

[[0, 67, 9, 1, 1],
 [0, 6702, 7, 1, 4],
 [0, 242, 10, 1, 4],
 [0, 4898, 0, 1, 1],
 [0, 21, 10, 1, 0],
 [0, 24, 9, 1, 5],
 [0, 2104, 0, 1, 4],
 [0, 4722, 8, 1, 4],
 [0, 6098, 6, 1, 2],
 [0, 3125, 9, 1, 29],
 [0, 481, 10, 1, 79],
 [0, 68, 6, 2, 23],
 [0, 1689, 6, 2, 3],
 [0, 2913, 6, 2, 40],
 [0, 1250, 7, 2, 26],
 [0, 356, 9, 2, 24],
 [0, 121, 9, 2, 51],
 [0, 430, 9, 2, 1],
 [0, 1829, 7, 2, 1],
 [0, 1571, 10, 2, 25],
 [0, 578, 10, 2, 1],
 [0, 431, 8, 2, 1],
 [0, 2762, 9, 2, 24],
 [0, 570, 7, 2, 1],
 [0, 3418, 9, 2, 50],
 [0, 3010, 7, 2, 52],
 [0, 1004, 5, 2, 1],
 [0, 433, 6, 2, 1],
 [0, 600, 6, 2, 12],
 [0, 2034, 8, 2, 24],
 [0, 164, 8, 2, 1],
 [0, 4086, 6, 2, 51],
 [0, 457, 0, 2, 26],
 [0, 20, 0, 2, 220],
 [0, 1074, 0, 2, 1],
 [0, 761, 0, 2, 1],
 [0, 2248, 6, 2, 1],
 [0, 597, 0, 2, 1],
 [0, 1047, 6, 2, 1],
 [0, 459, 9, 2, 1],
 [0, 466, 7, 2, 1],
 [0, 2543, 7, 2, 52],
 [0, 419, 8, 2, 26],
 [0, 199, 8, 2, 1],
 [0, 169, 7, 2, 12],
 [0, 2547, 7, 2, 46],
 [0, 174, 4, 2, 24],
 [0, 448, 5, 2, 